In [ ]:
from sklearn.ensemble import IsolationForest

groupbyear = main.group_by(['아파트명','전용면적','year']).agg(pl.col('target').mean()).sort(['year','target'])

def detect_outliers(group):
    model = IsolationForest(contamination=0.005)
    preds = model.fit_predict(group.to_numpy().reshape(-1, 1))
    return [True if pred == -1 else False for pred in preds]

yearlist = []

for i in groupbyear.select(['year']).unique().to_series().to_list():
    val = groupbyear.filter(pl.col('year') == i).select(['target'])
    val = detect_outliers(val)
    yearlist.extend(val)

groupbyear = groupbyear.with_columns(
    is_outlier=pl.Series(yearlist)
)

In [ ]:
groupbyear.plot.tick(
    x='year:T',
    y='target',
    tooltip=['아파트명'],
    color = 'is_outlier'
)

In [ ]:
outliers = groupbyear.filter(pl.col('is_outlier') == True)['아파트명'].unique().to_list()
main = main.with_columns(
    pl.when(
        pl.col('아파트명').is_in(outliers)
    ).then(1)
    .otherwise(0)
    .alias('outliers')
)

test = test.with_columns(
    pl.when(
        pl.col('아파트명').is_in(outliers)
    ).then(1)
    .otherwise(0)
    .alias('outliers')
)